# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities = pd.read_csv("outputs/WeatherPy.csv", encoding="utf-8")
cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kloulklubed,7.0419,134.2556,86.00,66,75,13.80,PW,1614909355
1,Semey,50.4111,80.2275,23.00,93,75,20.13,KZ,1614909308
2,Mataura,-46.1927,168.8643,58.51,65,100,4.85,NZ,1614909357
3,Cape Town,-33.9258,18.4232,69.01,68,0,16.11,ZA,1614909249
4,Kingaroy,-26.5333,151.8333,80.60,44,40,6.91,AU,1614909357
...,...,...,...,...,...,...,...,...,...
576,Teacapan,22.5500,-105.7500,68.27,67,48,8.14,MX,1614910096
577,Meulaboh,4.1363,96.1285,81.05,81,2,3.85,ID,1614909827
578,Urucará,-2.5364,-57.7600,74.35,94,60,0.85,BR,1614910097
579,Diego de Almagro,-26.3667,-70.0500,59.88,62,0,1.12,CL,1614910097


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Assign needed variables
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]

In [5]:
# Generate heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
chosen_city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Wind Speed"] < 10) & (cities["Cloudiness"] == 0), :]
chosen_city_df = chosen_city_df.dropna(how='any')
chosen_city_df.reset_index(inplace=True)
del chosen_city_df['index']
chosen_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Sur,22.5667,59.5289,76.60,42,0,3.15,OM,1614909481
1,Saint-Pierre,-21.3393,55.4781,77.00,78,0,4.61,RE,1614909490
2,Saint-Philippe,-21.3585,55.7679,77.00,78,0,4.61,RE,1614909206
3,Juan de Ayolas,-27.4000,-56.9000,71.74,90,0,4.21,PY,1614909506
4,Teknāf,20.8624,92.3058,72.32,76,0,4.29,BD,1614909595
5,Ahipara,-35.1667,173.1667,70.81,58,0,7.81,NZ,1614909663
6,Sawtell,-30.3833,153.1000,78.80,61,0,9.22,AU,1614909676
7,Vila Velha,-20.3297,-40.2925,78.01,78,0,4.61,BR,1614909642
8,Barra Patuca,15.8000,-84.2833,76.24,79,0,7.05,HN,1614909682
9,Matara,5.9485,80.5353,75.49,79,0,2.26,LK,1614909686


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
